In [20]:
import itertools
import numpy as np
import pandas as pd

## Task 1
Find the frequent itemsets from the given dataset. Minsup = 0.15 and minconf = 0.80


In [21]:
data = {'Transactions':['A, B, C, D, F', 'A, B, C, D', ' A, B, C, D', 'A, B', 'B, C, E']}
df = pd.DataFrame(data, index=[1,2,3,4,5])
print(df)
items = ['A', 'B', 'C', 'D', 'E', 'F']

    Transactions
1  A, B, C, D, F
2     A, B, C, D
3     A, B, C, D
4           A, B
5        B, C, E


Their are six unique items in the data set including A, B, C, D, E and F.


In [22]:
c_1 = list(itertools.combinations(items, 1))

### First Iteration
For our first iteration we count the number of items of $ C_1 $ and find that none of our sets are below the minimum support threshold. Thus, none are pruned.

In [23]:
c_1 = {'Itemset': ['{A}', '{B}', '{C}', '{D}', '{E}', '{F}'], 'sup': [4/5, 5/5, 4/5, 3/5, 1/5, 1/5]}
print(pd.DataFrame(c_1).to_string(index=False))

Itemset  sup
    {A}  0.8
    {B}  1.0
    {C}  0.8
    {D}  0.6
    {E}  0.2
    {F}  0.2


### Second Iteration

In [24]:
c_2 = list(itertools.combinations(items, 2))